In [1]:
import time
import serial
import csv
import numpy as np

def sendcmd(command, resp=True):
    ser = serial.Serial(
        port='COM6',
        baudrate=57600,
        parity=serial.PARITY_ODD,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.SEVENBITS
    )
    if ser.isOpen() == 0:
        ser.open()
    comando = command + '\r\n'
    #print('comando: ' + comando)
    ser.write(comando.encode())  # Encode the command as bytes before writing
    out = b''  # Initialize out as bytes
    time.sleep(0.5)  # Wait one second before reading output
    while ser.inWaiting() > 0:
        out += ser.read(1)
    ser.close()
    return out.decode()  # Decode bytes into a string before returning

def PID(timeStamp, temp, setpoint, error, intError, gradient, K_P, T_I, T_D, 
        P_out, I_out, D_out, PID_out, M_out):
    error.append(setpoint[-1] - temp[-1])
    P_out.append(K_P * error[-1])
    
    if len(intError) < 1:
        intError.append(error[-1])
    else:
        intError.append(intError[-1] + error[-1])
    I_out.append((K_P/T_I) * intError[-1])
    
    if len(temp) >= 2:
        gradient.append((temp[-1] - temp[-2]) 
                        / (timeStamp[-1] - timeStamp[-2]))
        D_out.append(-(K_P*T_D) * gradient[-1])
    else:
        D_out.append(0.0)
    PID_out.append(P_out[-1] + I_out[-1] + D_out[-1])
    if PID_out[-1] > 60.0:
        M_out.append(60.0)
    elif PID_out[-1] <= 0.0:
        M_out.append(0.0)
    else:
        M_out.append(PID_out[-1])
    return M_out

def findSetpoint(timeStamp, rampRate, target, T0):
    sp = timeStamp * rampRate + T0
    if sp > target:
        sp = target
    return sp
        
#setpoint = [float(sendcmd('KRDG? A'))]

target = 130; rampRate = 0.75/60 
timeStep = 1
timeStamp = []; temp = []; setpoint = []
P_out = []; I_out = []; D_out = []; PID_out = []; M_out = []
error = []; intError = []; gradient = [None]
K_P = 1.768; T_I = 848.7; T_D = 0.0

filename = 'PIDdata6.csv'
fileheaders = ['time', 'temp', 'setpoint', 'M_out', 'P_out', 'I_out', 'D_out', 
               'PID_out', 'error', 'intError', 'gradient']     
with open(filename, 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile) 
    writer.writerow(fileheaders)  

startTime = time.time()
startTemp = float(sendcmd('KRDG? A'))
while True:
    timeStamp.append(time.time() - startTime)
    temp.append(float(sendcmd('KRDG? A')))
    #setpoint.append(findSetpoint(timeStamp[-1], rampRate, target, startTemp))
    setpoint.append(target)
    #if temp[-1] >= 270.0:
    #    sendcmd('MOUT 1, 0.0')
    #    print('Overheating, turning off heater.')
    #    break
    M_out = PID(timeStamp, temp, setpoint, error, intError, gradient, 
                K_P, T_I, T_D, P_out, I_out, D_out, PID_out, M_out)
    sendcmd('MOUT 1, ' + str(M_out[-1]))
    dataRow = [timeStamp[-1], temp[-1], setpoint[-1],
               M_out[-1], P_out[-1], I_out[-1], D_out[-1], PID_out[-1], 
               error[-1], intError[-1], gradient[-1]]
    with open(filename, 'a', newline='') as csvfile:  
        writer = csv.writer(csvfile)    
        writer.writerow(dataRow)
    print("time: ", round(timeStamp[-1],2), 
          " temp: ", round(temp[-1],2), 
          " setpoint: ", round(setpoint[-1],2),
          " error: ", round(error[-1],2),
          " M_out: ", round(M_out[-1],2), 
          " PID_out: ", round(PID_out[-1], 2),
         end='\r')
    time.sleep(timeStep)

ValueError: could not convert string to float: ''

In [3]:
import time
import serial
import csv
def sendcmd(command, resp=True):
    ser = serial.Serial(
        port='COM6',
        baudrate=57600,
        parity=serial.PARITY_ODD,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.SEVENBITS
    )

    if ser.isOpen() == 0:
        print('ndoikoi la nde rekaka\n')
        ser.open()
    comando = command + '\r\n'
    print('comando: ' + comando)
    ser.write(comando.encode())  # Encode the command as bytes before writing
    out = b''  # Initialize out as bytes
    time.sleep(0.5)  # Wait one second before reading output
    while ser.inWaiting() > 0:
        out += ser.read(1)
    ser.close()
sendcmd('MOUT 1, 0.0')

SerialException: could not open port 'COM6': PermissionError(13, 'Access is denied.', None, 5)

In [ ]:
#dataArray = np.array(dataList).T
#print('temp [K] A = ' + sendcmd('KRDG? A'))  # No need to use str() here
#time.sleep(0.2)
#print('Manual Output Command = ' + sendcmd('MOUT 1,0.0'))
#time.sleep(0.2)
#print('Heater 1 power [%] = ' + sendcmd('HTR? 1'))
#time.sleep(0.2)
#print('Temp SET POINT [K] = ' + sendcmd('SETP? 1'))
